# Data preparation

In [2]:
import pandas as pd
import numpy as np 

In [3]:
path = '../../data/' 
hotels = pd.read_csv(path + '/features_hotels.csv')
data = pd.read_csv(path + 'data.csv')
data_test = pd.read_csv(path + 'test_set.csv')

In [4]:
to_exclude = data[['city', 'language','mobile','request_number','date']].drop_duplicates()
# to_exclude.to_csv('to_exclude.csv', index = False)
print('Nombre de requêtes uniques :', to_exclude.shape[0])

Nombre de requêtes uniques : 8329


In [5]:
tot = 0 
for i in range(7):
    tot += np.unique(data.loc[data.request_number == i].avatar_id.values).shape[0]

print('Estimation du nombre total de requêtes :', tot)

Estimation du nombre total de requêtes : 11171


In [6]:
# exclusion des avatar posant problème
# data = data.drop(index = data.loc[data.avatar_id == 134].index)
# to_drop = [108,134,135,136,137,138,141]
to_drop = [134]
data = data.loc[~data.avatar_id.isin(to_drop)]

In [7]:
data = data.drop(['avatar_id'], axis = 1).drop_duplicates()

In [8]:
# ajout des caractéristiques des hotels
data = data.merge(hotels, on=['hotel_id','city'])
data_test = data_test.merge(hotels, on=['hotel_id','city'])
data_test = data_test.sort_values('index').reset_index(drop=True).drop(['index'], axis = 1)

In [9]:
# création de la colonne request_number dans le test set
data_test['request_number'] = 1
for avatar in np.unique(data_test['avatar_id']):
    data_test.loc[data_test['avatar_id'] == avatar, 'request_number'] = data_test['order_requests'].loc[data_test['avatar_id']== avatar] - min(data_test['order_requests'].loc[data_test['avatar_id']== avatar])+1

In [10]:
# ajout de la variable ville == langue
lang_cit = True

if lang_cit == True: 
    dic_lang = {'amsterdam':'dutch', 'copenhagen':'danish', 'madrid':'spanish', 'paris':'french', 'rome':'italian', 'sofia':'bulgarian', 'valletta':'maltese', 'vienna':'austrian' ,'vilnius':'lithuanian'}
    data['city_language'] = data['city'].map(dic_lang)
    data['is_same_cl'] = data['city_language']==data['language']
    data_test['city_language'] = data_test['city'].map(dic_lang)
    data_test['is_same_cl'] = data_test['city_language']==data_test['language']
#     colBool += ['is_same_cl']

In [11]:
col = ['city', 'date', 'language', 'mobile', 'request_number', 'stock', 'group', 'brand', 'parking', 'pool','children_policy', 'is_same_cl', 'price']
data = data[col]
data_test = data_test[col[:-1]]

In [12]:
def additive_smoothing(x,alpha= 1.96):
    """ alpha = 1.96 borne à 95% pour la loi normale  """
    moy = (x+alpha).mean()
    d = x.max()-x.min()
    N = x.count()
    return N*moy/(N+alpha*d)

additive_smoothing.__name__ = 'additive_smoothing'

In [13]:
colQT = ['request_number', 'stock', 'date']
notQT = ['city', 'language', 'mobile', 'group', 'brand', 'parking', 'pool','children_policy', 'is_same_cl'] 

In [14]:
for i in notQT:
    df_tmp = data[[i, 'price']].groupby(i).agg({'price': ['mean', 'var', additive_smoothing]}).price
    df_tmp = df_tmp.add_suffix('_' + i)
    colQT += list(df_tmp.columns.unique())
    data = data.join(df_tmp, on=i)
    data_test = data_test.join(df_tmp, on=i)

In [15]:
colQT += ['price']

In [16]:
df = data[colQT[:-1]]
df_test = data_test[colQT[:-1]]
y = data[colQT[-1]]

In [17]:
X_train = df
X_test = df_test
Y_train = y
Y_test = None

In [18]:
# pd.read_csv('test.csv')

# GBR

In [19]:
X_train

,request_number,stock,date,mean_city,var_city,additive_smoothing_city,mean_language,var_language,additive_smoothing_language,mean_mobile,...,additive_smoothing_parking,mean_pool,var_pool,additive_smoothing_pool,mean_children_policy,var_children_policy,additive_smoothing_children_policy,mean_is_same_cl,var_is_same_cl,additive_smoothing_is_same_cl
0,1,3,12,165.55173,5985.767282,166.516354,154.748637,5999.095160,150.843929,159.610475,...,162.905065,142.144634,4905.884746,143.917897,157.313797,6680.118848,159.092193,160.002779,6871.253349,161.778872
1,1,15,27,165.55173,5985.767282,166.516354,159.636337,6808.807107,158.623704,160.026551,...,162.905065,142.144634,4905.884746,143.917897,157.313797,6680.118848,159.092193,160.002779,6871.253349,161.778872
2,1,2,11,165.55173,5985.767282,166.516354,156.157476,6641.903925,155.367276,159.610475,...,162.905065,142.144634,4905.884746,143.917897,157.313797,6680.118848,159.092193,160.002779,6871.253349,161.778872
3,1,14,26,165.55173,5985.767282,166.516354,158.518461,6578.472846,154.689115,160.026551,...,162.905065,142.144634,4905.884746,143.917897,157.313797,6680.118848,159.092193,160.002779,6871.253349,161.778872
4,1,13,25,165.55173,5985.767282,166.516354,156.713749,6541.710854,152.944919,159.610475,...,162.905065,142.144634,4905.884746,143.917897,157.313797,6680.118848,159.092193,160.002779,6871.253349,161.778872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916700,1,84,39,165.55173,5985.767282,166.516354,164.508062,7283.720668,159.483584,159.610475,...,162.905065,142.144634,4905.884746,143.917897,157.313797,6680.118848,159.092193,160.002779,6871.253349,161.778872
916701,1,84,39,165.55173,5985.767282,166.516354,156.456098,6441.953525,153.253943,160.026551,...,162.905065,142.144634,4905.884746,143.917897,157.313797,6680.118848,159.092193,160.002779,6871.253349,161.778872
916702,1,110,44,165.55173,5985.767282,166.516354,165.404823,7131.140515,164.022882,160.026551,...,162.905065,142.144634,4905.884746,143.917897,157.313797,6680.118848,159.092193,160.002779,6871.253349,161.778872
916703,1,104,43,165.55173,5985.767282,166.516354,155.846236,6642.503216,152.507311,159.610475,...,162.905065,142.144634,4905.884746,143.917897,157.313797,6680.118848,159.092193,160.002779,6871.253349,161.778872


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

GBR = GradientBoostingRegressor()

GBR.set_params(learning_rate = .01,
               max_depth = 4,
               n_estimators = 1500,
               validation_fraction = .1,
               criterion = 'friedman_mse',
               subsample = .2,
               max_leaf_nodes = 50,
              max_features = 1.0,
              verbose = 1)

fit = GBR.fit(X_train,Y_train)

In [50]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

# Créez un modèle de Gradient Boosting
gbr = GradientBoostingRegressor()

# Définissez les paramètres à tester
param_grid = {'n_estimators': [50],
              'max_depth': [4],
              'min_samples_split': [6],
              'learning_rate': [.1],
              'subsample': [1],#[0.5, 0.8, 1.0],
              'max_features': [None],#[None, 'sqrt', 'log2'],
              'alpha': [.8,.9,1.],#[0.5, 1.0, 2.0],
              'criterion':['friedman_mse'],
              'validation_fraction' : [.2]#,[.1,.2,.3]
             }

# Créez un objet GridSearchCV
grid_search = GridSearchCV(gbr, param_grid, cv=4, return_train_score=True, verbose = True, n_jobs = -1)
grid_search.fit(X_train, Y_train)

Fitting 4 folds for each of 3 candidates, totalling 12 fits


/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
4 fits failed out of a total of 12.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/ensemble/_gb.py", line 577, in fit
    self._check_params()
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/ensemble/_gb.py", line 356, in _check_params
    check_scalar(
  File "/usr/local/insa/anacond

GridSearchCV(cv=4, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'alpha': [0.8, 0.9, 1.0],
                         'criterion': ['friedman_mse'], 'learning_rate': [0.1],
                         'max_depth': [4], 'max_features': [None],
                         'min_samples_split': [6], 'n_estimators': [50],
                         'subsample': [1], 'validation_fraction': [0.2]},
             return_train_score=True, verbose=True)

In [49]:
print(grid_search.best_params_)

{'alpha': 0.8, 'criterion': 'friedman_mse', 'learning_rate': 0.1, 'max_depth': 4, 'max_features': None, 'min_samples_split': 6, 'n_estimators': 50, 'subsample': 1, 'validation_fraction': 0.1}


In [ ]:
grid_search.score(X_test,)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

X, y = X_train, Y_train

GBR.set_params(learning_rate = .01,
               max_depth = 4,
               n_estimators = 1500,
               validation_fraction = .1,
               criterion = 'friedman_mse',
               subsample = .2,
               max_leaf_nodes = 50,
              max_features = 1.0,
              verbose = 1)


param_grid = {
    'learning_rate': np.linspace(.001,0.2,5),
    'max_depth' : [4],
    'n_estimators': [500],
    'validation_fraction': [.1,.2,.3], # learning rate
    'criterion':['friedman_mse'],
    'subsample': np.linspace(.1,.6,5),
    'max_leaf_nodes' :  np.linspace(40,150,5),
    'max_features' : np.linspace(1.,5.,5),
}


GBR = GradientBoostingRegressor()


random_search = RandomizedSearchCV(GBR,
                                   param_grid,
                                   n_iter=5,
                                   cv=6,
                                   scoring='accuracy',
                                   random_state=0,
                                   verbose=True)

random_search.fit(X, y)

print(random_search.best_params_)

In [ ]:
# GBR.get_params().keys()

In [ ]:
y_pred = fit.predict(X_test)

In [ ]:
y_pred

In [ ]:
# from joblib import dump, load
# dump(GBR, 'model/target_encoding.pkl')

# Submit

In [ ]:
predictions = y_pred

In [ ]:
# save
_round = False # tester aussi des arrondis au dessus ou en dessous

name = 'target_encoding_1500_estim'

if _round : y_pred = np.round(predictions)
else : y_pred = predictions.copy()

sub = pd.DataFrame(y_pred)
sub.to_csv(path + 'submit/' + name + '.csv',index=True, header=['price'], index_label = 'index')